In [9]:
## RAG Application With GPT-4o
!pip install -r requirements.txt

  Obtaining dependency information for langchain_openai from https://files.pythonhosted.org/packages/95/ac/ec2eaa55a7eea519b17a7687e5bf1387dd313f41cf19750f9cfb96db44f9/langchain_openai-0.1.8-py3-none-any.whl.metadata
  Obtaining dependency information for langchain_core from https://files.pythonhosted.org/packages/aa/e2/9d7ccae2e2b2983912d71eacf8626d7b5f186389cd82eaa2930896132feb/langchain_core-0.2.2-py3-none-any.whl.metadata
  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/0e/86/69fdac2ec6852304bda08e5af5b72dfa6e74dc0b3cef0d7c1e19994388ae/streamlit-1.35.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for langchain_community from https://files.pythonhosted.org/packages/a5/c9/7d5a501f96cb508bc55a00be5f7e38f6c7c9880b534b7e21a70a8e7ea277/langchain_community-0.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for langserve from https://files.pythonhosted.org/packages/97/92/ab9bc89750ee947b89e3c0b880ae39b24e05257

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
botocore 1.27.59 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.2.1 which is incompatible.
conda 23.7.2 requires ruamel-yaml<0.18,>=0.11.14, but you have ruamel-yaml 0.18.6 which is incompatible.
faster-whisper 1.0.0 requires tokenizers<0.16,>=0.13, but you have tokenizers 0.19.1 which is incompatible.
python-lsp-black 1.2.1 requires black>=22.3.0, but you have black 0.0 which is incompatible.


In [10]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

# 1. Get a Data Loader


In [11]:
from langchain_community.document_loaders import WebBaseLoader


In [13]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
data = loader.load()
data

[Document(page_content="\n\n\n\n\nLangSmith User Guide | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith\n\n\n\n\n\n\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, weâ€™ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if theyâ€™re just starting their journey.Prototypingâ€‹Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.

# 2. Convert data to Vector Database


In [4]:

from langchain_objectbox.vectorstores import ObjectBox ##vector Database
from langchain_openai import OpenAIEmbeddings


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(data)

In [6]:
documents

[Document(page_content='LangSmith User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}),
 Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform

In [14]:
from langchain_openai import OpenAIEmbeddings
vector = ObjectBox.from_documents(documents, OpenAIEmbeddings(), embedding_dimensions=768)
vector

NameError: name 'ObjectBox' is not defined

# 3. Make a RAG pipeline


In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

ModuleNotFoundError: No module named 'langchain'

In [16]:
llm = ChatOpenAI(model="gpt-4o") ## Calling Gpt-4o
prompt = hub.pull("rlm/rag-prompt")
prompt


NameError: name 'hub' is not defined

In [12]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [10]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vector.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [11]:
question = "Explain what is langsmith"
result = qa_chain({"query": question })
result

e:\Langchain\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'Explain what is langsmith',
 'result': 'LangSmith is a platform designed for the development, monitoring, and testing of LLM (Large Language Model) applications. It supports various stages of the application lifecycle, including prototyping, debugging, beta testing, and production. The platform offers tools for tracing, evaluation, feedback collection, A/B testing, and automations to ensure optimal performance and quality of LLM applications.'}

In [12]:
result["result"]

'LangSmith is a platform designed for the development, monitoring, and testing of LLM (Large Language Model) applications. It supports various stages of the application lifecycle, including prototyping, debugging, beta testing, and production. The platform offers tools for tracing, evaluation, feedback collection, A/B testing, and automations to ensure optimal performance and quality of LLM applications.'

In [13]:
import pprint
pp = pprint.PrettyPrinter(indent=5)
pp.pprint(result["result"])

('LangSmith is a platform designed for the development, monitoring, and '
 'testing of LLM (Large Language Model) applications. It supports various '
 'stages of the application lifecycle, including prototyping, debugging, beta '
 'testing, and production. The platform offers tools for tracing, evaluation, '
 'feedback collection, A/B testing, and automations to ensure optimal '
 'performance and quality of LLM applications.')


In [14]:
question = "Explain Monitoring and A/B Testing in langsmith"
result = qa_chain({"query": question })
result

{'query': 'Explain Monitoring and A/B Testing in langsmith',
 'result': 'LangSmith provides monitoring charts to track key metrics over time, allowing users to drill down into specific data points for debugging production issues. It also supports A/B testing by enabling tag and metadata grouping, which allows users to compare the performance of different versions of their applications side-by-side within each chart. This helps in evaluating changes in prompt, model, or retrieval strategy effectively.'}